## Passionate Investors. ERC20 Analysis

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML, display
import pandas as pd

from config import PROJECT_ID, INITIAL_TS, SNAPSHOT_TS, \
    ERC20_ANALYSIS_DATASET_NAME, ERC20_BALANCES_TABLE_NAME, ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC20_TOKEN_METADATA_TABLE_NAME, bq_client
from utils import drop_table, create_table, create_table_from_df

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule

<table style="text-align: center">
    <thead style="text-align: center">
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> Number of tokens >= 2 and ERC20 balance </td>
            <td> > 0.01 ETH </td>
            <td> > 10 ETH </td>
            <td> > 100 ETH </td>
        </tr>
    </tbody>
</table>

In [2]:
min_number_of_tokens = 2
boundary_erc20_balance = [0.01, 10, 100]

### Set Token Data

In [3]:

erc20_tokens_at_1920000_block = ['0xbb9bc244d798123fde783fcc1c72d3bb8c189413']
erc20_tokens_at_1920000_block = [item.lower() for item in erc20_tokens_at_1920000_block]

# TODO check decimals from parity data
erc20_tokens_dict = {
               # name:    [address,                                      ETH price,  decimals]
                'YFI':    ['0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e', 20.70,      18],
                'FTM':    ['0x4e15361fd6b4bb609fa63c81a2be19d873717870', 0.000234,   18],
                'AMPL':   ['0xd46ba6d942050d489dbd938a2c909a5d5039a161', 0.0005178,   9],
                'renBTC': ['0xeb4c2781e4eba804ce9a9803c67d0893436bb27d', 32.216776,   8],
                'OCEAN':  ['0x967da4048cd07ab37855c090aaf366e4ce1b9f48', 0.000747,   18],
                'RPL':    ['0xb4efd85c19999d84251304bda99e90b92300bd93', 0.006481,   18],
                'UNI':    ['0x1f9840a85d5af5bf1d1762f925bdaddc4201f984', 0.01733,    18],
                'KEEP3R': ['0x1ceb5cb57c4d4e2b2433641b95dd330a33185a44', 0.1349,     18],
                'GST2':   ['0x0000000000b3f879cb30fe243b4dfee438691c04', 0.1671,      2],
                'CHI':    ['0x0000000000004946c0e9f43f4dee607b0ef1fa1c', 0.002281,    0],
                'MLN':    ['0xec67005c4e498ec7f55e092bd1d35cbc47c91892', 0.02349,    18],
                'AAVE':   ['0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 0.2248,     18],
                'GNT':    ['0xa74476443119A942dE498590Fe1f2454d7D4aC0d', 0.0002534,  18],
                'GLM':    ['0x7dd9c5cba05e151c895fde1cf355c9a1d5da6429', 0.0002536,  18],
                'ZRX':    ['0xe41d2489571d322189246dafa5ebde1f4699f498', 0.0007704,  18],
                'SNX':    ['0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f', 0.01195,    18],
                'SUSHI':  ['0x6b3595068778dd592e39a122f4f5a5cf09c90fe2', 0.009907,   18],
                'COMP':   ['0xc00e94cb662c3520282e6f5717214004a7f26888', 0.2611,     18],
                'BAL':    ['0xba100000625a3754423978a60c9317c58a424e3d', 0.02504,    18],
                'GRT':    ['0xc944e90c64b2c07662a292be6244bdf05cda44a7', 0.001134,   18],
                'MKR':    ['0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2', 1.2219,     18],
                'CRV':    ['0xd533a949740bb3306d119cc777fa900ba034cd52', 0.001204,   18],
                'LPT':    ['0xb6c4267c4877bb0d6b1685cfd85b0fbe82f105ec', 0.002672,   18],
                'REL':    ['0xb6c4267c4877bb0d6b1685cfd85b0fbe82f105ec', 0.0005475,  18],
                'ANT':    ['0xa117000000f279d81a1d3cc75430faa017fa5a2e', 0.002823,   18],
                'GEN':    ['0x543ff227f64aa17ea132bf9886cab5db55dcaddf', 0.00009676, 18],
                'PNK':    ['0x93ed3fbe21207ec2e8f2d3c3de6e058cb73bc04d', 0.00005532, 18],
                'REP':    ['0x221657776846890989a759ba2973e427dff5c9bb', 0.0182,     18],
                'SNT':    ['0x744d70fdbe2ba4cf95131626614a1763df805b9e', 0.00005182, 18],
                '2KEY':   ['0xE48972fCd82a274411c01834e2f031D4377Fa2c0', 0.00003234, 18],
                'NU':     ['0x4fE83213D56308330EC302a8BD641f1d0113A4Cc', 0.0003783,  18],
                'MANA':   ['0x0f5d2fb29fb7d3cfee444a200298f468908cc942', 0.0002821,  18],
                'FOAM':   ['0x4946fcea7c692606e8908002e55a582af44ac121', 0.00004279, 18],
                'XRT':    ['0x7de91b204c1c737bcee6f000aaa6569cf7061cb7', 0.01895,     9],
                'GOLD':   ['0x150b0b96933b75ce27af8b92441f8fb683bf9739', 0.00000265, 18],
                'TORN':   ['0x77777feddddffc19ff86db637967013e6c6a116c', 0.08169,    18]}

erc20_tokens_dict = {k: [v[0].lower(), v[1], v[2]] for k, v in erc20_tokens_dict.items()}
erc20_tokens_addresses_tuple = tuple([item[0] for item in erc20_tokens_dict.values()])
erc20_tokens_df = pd.DataFrame(erc20_tokens_dict.values(), index=erc20_tokens_dict.keys(), columns=['address', 'eth_price', 'decimals']).reset_index()

### Create Token Metadata Table

In [4]:
if CREATE_TABLES:
    r = create_table_from_df(erc20_tokens_df,
                             ERC20_TOKEN_METADATA_TABLE_NAME,
                             ERC20_ANALYSIS_DATASET_NAME)

Table erc20_analysis:erc20_token_metadata has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/erc20_analysis/tables/erc20_token_metadata?prettyPrint=false: Not found: Table cosmic-keep-223223:erc20_analysis.erc20_token_metadata


### Get Balances

In [5]:
query_1 = f'''
        WITH token_transfers AS (
                SELECT
                    token_address,
                    address,
                    sum(value) as balance
                FROM (SELECT
                            token_address,
                            from_address as address,
                            - safe_cast(value as numeric) as value
                        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
                        WHERE token_address IN {erc20_tokens_addresses_tuple}
                          AND from_address NOT IN ('0x0000000000000000000000000000000000000000')
                          AND block_timestamp >= '{INITIAL_TS}'
                          AND block_timestamp <= '{SNAPSHOT_TS}'

                        UNION ALL

                        SELECT
                            token_address,
                            to_address as address,
                            safe_cast(value as numeric) as value
                        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
                        WHERE token_address IN {erc20_tokens_addresses_tuple}
                          AND to_address NOT IN ('0x0000000000000000000000000000000000000000')
                          AND block_timestamp >= '{INITIAL_TS}'
                          AND block_timestamp <= '{SNAPSHOT_TS}'
                        )
                GROUP BY token_address, address
        )
        SELECT
            token_address,
            token_transfers.address as address,
            balance,
            token_metadata.eth_price as eth_price,
            token_metadata.decimals as decimals,
            balance * token_metadata.eth_price / pow(10, token_metadata.decimals) as eth_balance
        FROM token_transfers
        LEFT JOIN `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_TOKEN_METADATA_TABLE_NAME}` as token_metadata
        ON token_transfers.token_address = token_metadata.address
'''

if DROP_TABLES:
    drop_table(ERC20_BALANCES_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

if CREATE_TABLES:
    create_table(query_1, ERC20_BALANCES_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

Table erc20_analysis:erc20_balances has been deleted.
Table erc20_analysis:erc20_balances has been created and filled 3943624 rows.


### Create Distribution Table

In [6]:
query_2 = f'''
SELECT
    address,
    CASE
        WHEN sum_eth_balance > 100  THEN 3
        WHEN sum_eth_balance > 10   THEN 2
        WHEN sum_eth_balance > 0.01 THEN 1
    END
    AS grade,
    sum_eth_balance,
    number_of_tokens
FROM (
    SELECT
        address,
        count( distinct token_address) as number_of_tokens,
        sum(eth_balance) as sum_eth_balance
    FROM `cosmic-keep-223223.erc20_analysis.erc20_balances`
    WHERE balance > 0
    GROUP BY address)
WHERE number_of_tokens >=2
'''

if DROP_TABLES:
    drop_table(ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

if CREATE_TABLES:
    create_table(query_2, ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

Table erc20_analysis:erc20_analysis_distribution has been deleted.
Table erc20_analysis:erc20_analysis_distribution has been created and filled 160374 rows.
